In [1]:
%pip install langchain-openai langchain-core python-dotenv


[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
client = OpenAI()

In [6]:
user_message = "What have I asked before?"

In [7]:
response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Answer all user questions"},
            {"role": "user", "content": user_message},
        ],
        temperature=0.0,
    )
response.choices[0].message.content

"I'm sorry, but I don't have the ability to recall past interactions or access previous conversations. Each session is treated independently for privacy and security reasons. How can I assist you today?"

In [8]:
memory = [
    {"role": "system", "content": "Answer all user questions"},
    {"role": "user", "content": "What's an API?"},
]

In [9]:
new_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=memory,
    temperature=0.0,
)

memory.append(
    {"role": "assistant", "content": new_response.choices[0].message.content}
)

memory

[{'role': 'system', 'content': 'Answer all user questions'},
 {'role': 'user', 'content': "What's an API?"},
 {'role': 'assistant',
  'content': 'An API, or Application Programming Interface, is a set of rules and protocols that allows different software applications to communicate with each other. It defines the methods and data formats that applications can use to request and exchange information. APIs enable developers to access the functionality of other software services, libraries, or platforms without needing to understand their internal workings.\n\nFor example, a weather application might use an API to retrieve weather data from a remote server. The API specifies how the application can request the data, what parameters it can send, and the format in which the data will be returned. This allows developers to build applications that can leverage existing services and data, facilitating integration and enhancing functionality.'}]

In [10]:
memory.append(
    {"role": "user", "content": "What have I asked?"}
)

memory

[{'role': 'system', 'content': 'Answer all user questions'},
 {'role': 'user', 'content': "What's an API?"},
 {'role': 'assistant',
  'content': 'An API, or Application Programming Interface, is a set of rules and protocols that allows different software applications to communicate with each other. It defines the methods and data formats that applications can use to request and exchange information. APIs enable developers to access the functionality of other software services, libraries, or platforms without needing to understand their internal workings.\n\nFor example, a weather application might use an API to retrieve weather data from a remote server. The API specifies how the application can request the data, what parameters it can send, and the format in which the data will be returned. This allows developers to build applications that can leverage existing services and data, facilitating integration and enhancing functionality.'},
 {'role': 'user', 'content': 'What have I asked?'

In [11]:
new_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=memory,
    temperature=0.0,
)

memory.append(
    {"role": "assistant", "content": new_response.choices[0].message.content}
)

memory

[{'role': 'system', 'content': 'Answer all user questions'},
 {'role': 'user', 'content': "What's an API?"},
 {'role': 'assistant',
  'content': 'An API, or Application Programming Interface, is a set of rules and protocols that allows different software applications to communicate with each other. It defines the methods and data formats that applications can use to request and exchange information. APIs enable developers to access the functionality of other software services, libraries, or platforms without needing to understand their internal workings.\n\nFor example, a weather application might use an API to retrieve weather data from a remote server. The API specifies how the application can request the data, what parameters it can send, and the format in which the data will be returned. This allows developers to build applications that can leverage existing services and data, facilitating integration and enhancing functionality.'},
 {'role': 'user', 'content': 'What have I asked?'

In [12]:
from typing import List, Dict, Literal

In [ ]:
class Memory:
    def __init__(self):
        self.messages: List[Dict[str, str]] = []

    def add_message(self, role: Literal['user', 'system', 'assistant'], content: str):
        self.messages.append({
            "role": role,
            "content": content
        })

    def get_messages(self) -> List[Dict[str, str]]:
        return self.messages

In [ ]:
def chat(user_message:str=None, memory:Memory=None)->str:
    messages = [{"role": "user", "content": user_message}]
    if memory:
        if user_message:
            memory.add_message(role="user", content=user_message)
        messages = memory.get_messages()

    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        temperature = 0.0,
        messages = messages,
    )

    ai_message = response.choices[0].message.content
    if memory:
        memory.add_message(role="assistant", content=ai_message)

    return ai_message

In [15]:
memory = Memory()


In [16]:
memory.add_message(role="system", content="You're a helpful assitant")

In [17]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant"}]

In [18]:
chat(
    user_message = "what's the capital of Brazil",
)

'The capital of Brazil is Brasília. It was officially inaugurated as the capital on April 21, 1960, and was designed by the architect Oscar Niemeyer and urban planner Lúcio Costa.'

In [19]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant"}]

In [20]:
chat(
    user_message = "what's the capital of Brazil",
    memory=memory
)

'The capital of Brazil is Brasília. It was officially inaugurated as the capital on April 21, 1960, and was designed by the architect Oscar Niemeyer and urban planner Lúcio Costa.'

In [21]:
chat(
    user_message = "what have I asked?",
    memory=memory
)

'You asked about the capital of Brazil.'

In [22]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant"},
 {'role': 'user', 'content': "what's the capital of Brazil"},
 {'role': 'assistant',
  'content': 'The capital of Brazil is Brasília. It was officially inaugurated as the capital on April 21, 1960, and was designed by the architect Oscar Niemeyer and urban planner Lúcio Costa.'},
 {'role': 'user', 'content': 'what have I asked?'},
 {'role': 'assistant', 'content': 'You asked about the capital of Brazil.'}]